In [193]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)



In [194]:
data = pd.read_csv('Environment_Temperature_change_E_All_Data.csv', encoding='ISO-8859-1')
# Identify columns that start with 'Y'
# Rename columns that start with 'Y' by removing the 'Y' character
new_columns = {col: col[1:] if col.startswith('Y') else col for col in data.columns}
data.rename(columns=new_columns, inplace=True)


data

,Area Code,Area Code (M49),Area,Months Code,Months,Element Code,Element,Unit,1961,1961F,1962,1962F,1963,1963F,1964,1964F,1965,1965F,1966,1966F,1967,1967F,1968,1968F,1969,1969F,1970,1970F,1971,1971F,1972,1972F,1973,1973F,1974,1974F,1975,1975F,1976,1976F,1977,1977F,1978,1978F,1979,1979F,1980,1980F,1981,1981F,1982,1982F,1983,1983F,1984,1984F,1985,1985F,1986,1986F,1987,1987F,1988,1988F,1989,1989F,1990,1990F,1991,1991F,1992,1992F,1993,1993F,1994,1994F,1995,1995F,1996,1996F,1997,1997F,1998,1998F,1999,1999F,2000,2000F,2001,2001F,2002,2002F,2003,2003F,2004,2004F,2005,2005F,2006,2006F,2007,2007F,2008,2008F,2009,2009F,2010,2010F,2011,2011F,2012,2012F,2013,2013F,2014,2014F,2015,2015F,2016,2016F,2017,2017F,2018,2018F,2019,2019F,2020,2020F,2021,2021F,2022,2022F
0,2,'004,Afghanistan,7001,January,7271,Temperature change,°C,0.756,E,0.030,E,2.716,E,-5.260,E,1.834,E,3.780,E,-1.398,E,0.349,E,-2.325,E,0.765,E,-1.527,E,-1.299,E,-2.929,E,-1.194,E,-0.473,E,2.430,E,-3.001,E,-0.651,E,1.138,E,-0.505,E,1.725,E,0.715,E,0.621,E,-0.106,E,0.528,E,0.345,E,2.689,E,1.223,E,-1.127,E,0.530,E,0.031,E,0.607,E,-0.734,E,1.131,E,1.287,E,-0.720,E,1.402,E,0.953,E,0.871,E,1.577,E,-0.592,E,1.839,E,2.447,E,2.657,E,0.093,E,-1.489,E,0.499,E,-5.684,E,1.535,E,3.619,E,1.233,E,-0.504,E,1.331,E,1.698,E,1.908,E,3.444,E,1.256,E,2.051,E,2.858,E,-0.681,E,0.559,E,2.284,E
1,2,'004,Afghanistan,7001,January,6078,Standard Deviation,°C,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E,1.964,E
2,2,'004,Afghanistan,7002,February,7271,Temperature change,°C,-1.766,E,2.414,E,3.888,E,-0.236,E,-0.130,E,3.514,E,0.160,E,-2.102,E,-3.209,E,1.756,E,0.831,E,-7.713,E,1.854,E,-3.688,E,-1.224,E,-1.617,E,-0.169,E,-0.378,E,1.090,E,-1.194,E,1.091,E,-2.071,E,0.454,E,-4.295,E,2.550,E,-0.210,E,1.428,E,0.240,E,-2.898,E,-0.357,E,-1.382,E,-0.099,E,1.507,E,-1.358,E,-0.161,E,0.865,E,0.025,E,-0.530,E,3.237,E,-0.922,E,0.698,E,0.961,E,1.747,E,3.087,E,-1.567,E,4.826,E,1.660,E,-2.306,E,2.454,E,1.241,E,0.211,E,-3.068,E,1.558,E,-3.108,E,2.585,E,2.403,E,-0.224,E,2.714,E,0.067,E,2.500,E,4.086,E,2.371,E
3,2,'004,Afghanistan,7002,February,6078,Standard Deviation,°C,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E,2.602,E
4,2,'004,Afghanistan,7003,March,7271,Temperature change,°C,0.458,E,1.277,E,0.363,E,1.618,E,-0.947,E,-0.021,E,-0.663,E,0.443,E,2.429,E,-0.951,E,1.667,E,-1.770,E,-0.482,E,1.002,E,-0.575,E,-2.806,E,3.319,E,-1.557,E,-1.389,E,-0.548,E,1.160,E,-1.799,E,-2.362,E,1.792,E,-0.154,E,-3.052,E,1.349,E,0.026,E,0.074,E,-1.093,E,-0.904,E,-2.174,E,-1.427,E,1.291,E,-1.467,E,-0.367,E,0.011,E,-0.609,E,-0.229,E,-0.292,E,1.220,E,1.990,E,-0.159,E,2.837,E,1.688,E,1.945,E,-0.105,E,4.324,E,2.411,E,3.464,E,0.822,E,-0.384,E,2.324,E,-0.051,E,-0.553,E,2.388,E,0.877,E,4.533,E,0.293,E,0.555,E,2.480,E,3.598,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Drop columns that end with F and Area codes

In [195]:
columns_to_drop = [col for col in data.columns if col.endswith('F')] 
columns_to_drop +=['Area Code', 'Area Code (M49)' , 'Months Code','Element Code','Unit']
data = data.drop(columns=columns_to_drop)
data

,Area,Months,Element,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,January,Temperature change,0.756,0.030,2.716,-5.260,1.834,3.780,-1.398,0.349,-2.325,0.765,-1.527,-1.299,-2.929,-1.194,-0.473,2.430,-3.001,-0.651,1.138,-0.505,1.725,0.715,0.621,-0.106,0.528,0.345,2.689,1.223,-1.127,0.530,0.031,0.607,-0.734,1.131,1.287,-0.720,1.402,0.953,0.871,1.577,-0.592,1.839,2.447,2.657,0.093,-1.489,0.499,-5.684,1.535,3.619,1.233,-0.504,1.331,1.698,1.908,3.444,1.256,2.051,2.858,-0.681,0.559,2.284
1,Afghanistan,January,Standard Deviation,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964
2,Afghanistan,February,Temperature change,-1.766,2.414,3.888,-0.236,-0.130,3.514,0.160,-2.102,-3.209,1.756,0.831,-7.713,1.854,-3.688,-1.224,-1.617,-0.169,-0.378,1.090,-1.194,1.091,-2.071,0.454,-4.295,2.550,-0.210,1.428,0.240,-2.898,-0.357,-1.382,-0.099,1.507,-1.358,-0.161,0.865,0.025,-0.530,3.237,-0.922,0.698,0.961,1.747,3.087,-1.567,4.826,1.660,-2.306,2.454,1.241,0.211,-3.068,1.558,-3.108,2.585,2.403,-0.224,2.714,0.067,2.500,4.086,2.371
3,Afghanistan,February,Standard Deviation,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602
4,Afghanistan,March,Temperature change,0.458,1.277,0.363,1.618,-0.947,-0.021,-0.663,0.443,2.429,-0.951,1.667,-1.770,-0.482,1.002,-0.575,-2.806,3.319,-1.557,-1.389,-0.548,1.160,-1.799,-2.362,1.792,-0.154,-3.052,1.349,0.026,0.074,-1.093,-0.904,-2.174,-1.427,1.291,-1.467,-0.367,0.011,-0.609,-0.229,-0.292,1.220,1.990,-0.159,2.837,1.688,1.945,-0.105,4.324,2.411,3.464,0.822,-0.384,2.324,-0.051,-0.553,2.388,0.877,4.533,0.293,0.555,2.480,3.598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9651,OECD,JunJulAug,Standard Deviation,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247,0.247
9652,OECD,SepOctNov,Temperature change,0.044,0.467,0.667,-0.152,-0.197,-0.292,0.324,0.217,-0.028,-0.150,-0.001,-0.637,0.129,-0.552,-0.131,-0.538,0.370,-0.456,0.529,0.322,0.583,-0.095,0.680,-0.283,-0.519,-0.492,0.687,0.296,0.025,0.224,0.054,-0.371,-0.373,0.640,0.302,-0.182,0.594,1.306,0.838,0.545,0.838,0.743,0.916,0.756,1.252,0.997,0.925,0.804,1.107,0.965,1.118,0.896,1.066,1.036,1.708,1.555,1.226,0.598,1.247,1.400,1.582,1.204
9653,OECD,SepOctNov,Standard Deviation,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380,0.380
9654,O

In [196]:
data['Months'].value_counts()


Months
January                568
October                568
SepOctNov            568
JunJulAug            568
MarAprMay            568
DecJanFeb            568
December               568
November               568
September              568
February               568
August                 568
July                   568
June                   568
May                    568
April                  568
March                  568
Meteorological year    568
Name: count, dtype: int64

In [197]:
season_mapping = {
    'Dec\x96Jan\x96Feb': 'Winter',
    'Mar\x96Apr\x96May': 'Spring',
    'Jun\x96Jul\x96Aug': 'Summer',
    'Sep\x96Oct\x96Nov': 'Fall'
}

data.replace(season_mapping, inplace=True)
data['Months'].value_counts()



Months
January                568
October                568
Fall                   568
Summer                 568
Spring                 568
Winter                 568
December               568
November               568
September              568
February               568
August                 568
July                   568
June                   568
May                    568
April                  568
March                  568
Meteorological year    568
Name: count, dtype: int64

In [198]:
data.head()

,Area,Months,Element,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,January,Temperature change,0.756,0.030,2.716,-5.260,1.834,3.780,-1.398,0.349,-2.325,0.765,-1.527,-1.299,-2.929,-1.194,-0.473,2.430,-3.001,-0.651,1.138,-0.505,1.725,0.715,0.621,-0.106,0.528,0.345,2.689,1.223,-1.127,0.530,0.031,0.607,-0.734,1.131,1.287,-0.720,1.402,0.953,0.871,1.577,-0.592,1.839,2.447,2.657,0.093,-1.489,0.499,-5.684,1.535,3.619,1.233,-0.504,1.331,1.698,1.908,3.444,1.256,2.051,2.858,-0.681,0.559,2.284
1,Afghanistan,January,Standard Deviation,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964,1.964
2,Afghanistan,February,Temperature change,-1.766,2.414,3.888,-0.236,-0.130,3.514,0.160,-2.102,-3.209,1.756,0.831,-7.713,1.854,-3.688,-1.224,-1.617,-0.169,-0.378,1.090,-1.194,1.091,-2.071,0.454,-4.295,2.550,-0.210,1.428,0.240,-2.898,-0.357,-1.382,-0.099,1.507,-1.358,-0.161,0.865,0.025,-0.530,3.237,-0.922,0.698,0.961,1.747,3.087,-1.567,4.826,1.660,-2.306,2.454,1.241,0.211,-3.068,1.558,-3.108,2.585,2.403,-0.224,2.714,0.067,2.500,4.086,2.371
3,Afghanistan,February,Standard Deviation,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602,2.602
4,Afghanistan,March,Temperature change,0.458,1.277,0.363,1.618,-0.947,-0.021,-0.663,0.443,2.429,-0.951,1.667,-1.770,-0.482,1.002,-0.575,-2.806,3.319,-1.557,-1.389,-0.548,1.160,-1.799,-2.362,1.792,-0.154,-3.052,1.349,0.026,0.074,-1.093,-0.904,-2.174,-1.427,1.291,-1.467,-0.367,0.011,-0.609,-0.229,-0.292,1.220,1.990,-0.159,2.837,1.688,1.945,-0.105,4.324,2.411,3.464,0.822,-0.384,2.324,-0.051,-0.553,2.388,0.877,4.533,0.293,0.555,2.480,3.598


### Global average temperature change

In [211]:
temperature_change_rows = data[data['Element'] == 'Temperature change']
temperature_change_rows
global_data = temperature_change_rows.drop(columns=['Months', 'Area'])
global_data = global_data.groupby( 'Element').mean().reset_index()
global_data.rename(columns={'Element':'Year'}, inplace=True)
global_data



,Year,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Temperature change,0.145023,-0.030538,-0.017029,-0.115601,-0.220638,0.096785,-0.127709,-0.167658,0.111611,0.081577,-0.174006,-0.061713,0.199511,-0.120371,-0.017683,-0.20991,0.185727,0.053148,0.229429,0.217004,0.222635,0.161705,0.358265,0.097542,0.079285,0.139265,0.425695,0.445452,0.292074,0.580367,0.344281,0.262186,0.247242,0.574151,0.618407,0.308548,0.570011,0.949176,0.735716,0.681739,0.804913,0.92659,0.859787,0.783214,0.870102,0.894557,0.996986,0.808099,0.924945,1.060195,0.846288,0.88457,0.963813,1.113567,1.299679,1.422832,1.281094,1.299438,1.449905,1.5156,1.332076,1.362985


In [224]:
global_data_T = global_data.T
global_data_T.reset_index(inplace=True)
new_header = global_data_T.iloc[0] #grab the first row for the header
global_data_T = global_data_T[1:] #take the data less the header row
global_data_T.columns = new_header #set the header row as the df header
global_data_T['Year'] = global_data_T['Year'].astype(int)
global_data_T


,Year,Temperature change
1,1961,0.145023
2,1962,-0.030538
3,1963,-0.017029
4,1964,-0.115601
5,1965,-0.220638
...,...,...
58,2018,1.299438
59,2019,1.449905
60,2020,1.5156
61,2021,1.332076


In [229]:
fig = px.line(global_data_T, x='Year', y='Temperature change', title='Global Temperature Change (1961-2022)')
fig.show()